In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import toolplot
import tooltime

from ctc import evm
from ctc.protocols import fei_utils
from ctc.protocols import chainlink_utils

%matplotlib inline

In [ ]:
toolplot.setup_plot_formatting()

In [ ]:
# specify blocks

# start_block = 14000000
start_block = 13500000
end_block = await evm.async_get_latest_block_number()
n_samples = 1000

blocks = np.linspace(start_block, end_block, n_samples).astype(int)
print('start_block:', start_block)
print('end_block:', end_block)
print('total blocks:', end_block - start_block + 1)
print('blocks sampled:', len(blocks))

# Gather Data

In [ ]:
# gather pcv stats

pcv_stats = await fei_utils.async_get_pcv_stats_by_block(
    blocks=blocks,
    provider={'chunk_size': 1},
)

pcv_stats

In [ ]:
# gather prices

eth_usd = await chainlink_utils.async_get_eth_price_by_block(blocks=blocks)

eth_usd

In [ ]:
# compile data

data = pcv_stats.copy()
data['cr'] = data['pcv'] / data['user_fei']
data['eth_usd'] = eth_usd

In [ ]:
# save data

name = 'cadlabs_exploratory_data__' + tooltime.create_timestamp_label() + '.csv'
data.to_csv(name)
print('saved to: ' + name)

# Visualize Data

## metrics vs time

In [ ]:
plots = {
    'subplot_height': 4,
    'common': {
        'x': blocks,
        'name_position': 'ylabel',
        'tickgrid': True,
        'xtick_format': {'xticks_kwargs': {'color': 'white'}},
        'ytick_format': {},
    },
    'plots': {
        'ETH_USD': {
            'name': 'ETH_USD',
            'y': data['eth_usd'],
            'y_kwargs': {'color': 'mediumslateblue'},
            'title': 'Fei protocol metrics over time',
        },
        'PCV': {
            'name': 'PCV',
            'y': data['pcv'],
            'y_kwargs': {'color': 'green'},
        },
        'CR': {
            'name': 'Collateralization\nRatio',
            'y': data['cr'],
            'y_kwargs': {'color': 'green'},
        },
        'User FEI': {
            'name': 'User FEI',
            'y': data['user_fei'],
            'y_kwargs': {'color': 'green'},
            'xtick_format': {},
            'xlabel': 'block',
        },
    },
}

toolplot.plot_subplots(plots)

## metrics vs eth price

you can see that there is a strong linear correlation between PCV and ETH price

the other relationships are more complex and noisy

In [ ]:
for key, name in [
    ['pcv', 'PCV'],
    ['cr', 'Collateralization Ratio'],
    ['user_fei', 'User FEI'],
]:
    plt.scatter(eth_usd, data[key], c=blocks, cmap='nipy_spectral', s=70)
    plt.plot(eth_usd, data[key], color='black', alpha=0.1)
    cbar = plt.colorbar()
    cbar.set_label('block', labelpad=30, rotation=270)
    plt.xlabel('ETH_USD')
    plt.ylabel(name)
    plt.title(name + ' vs ' + str('ETH_USD'))
    toolplot.format_xticks()
    toolplot.format_yticks()
    toolplot.add_tick_grid()
    plt.show()